In [1]:
import pandas as pd
import numpy as np
import os
import trimesh
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import random

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# Load cellids into a list

cell_list = [648518346349539076, 648518346349538440, 648518346349539853, 648518346349537297, 648518346349539100, 648518346349539934, 648518346349537516]
len(cell_list)

7

In [4]:
# create vasculature list

celltypes = pd.read_csv("data/220309_cell_type_classification.csv", index_col=[0])
vasc = celltypes[celltypes.cell_type == 'vasculature']
vasc_list = vasc.cell_segid.to_list()
len(vasc_list)

72

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito.head()

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000


In [6]:
mito_list = mito[mito['cellid'].isin(cell_list)].mito_id.values.tolist()
len(mito_list)

4479

In [7]:
mito_list_dictionary_by_cell = {
    f'cell_{uid}': mito[mito.cellid == uid].mito_id.tolist()
    for uid in cell_list
}

# print(mito_list_dictionary_by_cell)

In [8]:
type(mito_list_dictionary_by_cell)

dict

In [9]:
mesh_dir = 'data/neuron_meshes_v185/dec/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/dec/'
mito_source = "precomputed://https://td.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [10]:
# generate trimesh objects of vascular and cell cells and place in a dictionary

def update_mesh_dictionary(cell_list, mesh_dictionary, prefix, mesh_dir):
    for cellid in cell_list:
        mesh_file = os.path.join(mesh_dir, str(cellid) + '.ply')
        mesh_dictionary[prefix + str(cellid)] = trimesh.load_mesh(mesh_file)

# Initialize the mesh dictionary
mesh_dictionary = {}

# Update the dictionary with cell_list
update_mesh_dictionary(cell_list, mesh_dictionary, 'cell_', mesh_dir)

# Update the dictionary with vasc_list
update_mesh_dictionary(vasc_list, mesh_dictionary, 'vasc_', mesh_dir)


In [11]:
len(mesh_dictionary)

24

In [12]:
# mito_list[:10]

In [13]:
# Create the nested dictionary of decimated mito meshes

trimesh_dictionary_by_cell = {}

for cell_key, mito_list in mito_list_dictionary_by_cell.items():
    # Initialize the second-level dictionary
    nested_dict = {}
    
    for mitoid in mito_list:
        # Construct the file path for the ply file
        ply_file = os.path.join(mito_mesh_dir, str(mitoid) + '.ply')
        
        # Debug: Print the file path to check correctness
        # print(f'Processing file: {ply_file}')
        
        # Check if the file exists and is accessible
        if not os.path.isfile(ply_file):
            print(f'Error: File does not exist - {ply_file}')
            continue
        
        try:
            # Load the ply file as a trimesh object
            mito_mesh = trimesh.load_mesh(ply_file)
            # Define the key for the second-level dictionary
            nested_key = f'trimesh_mito_{mitoid}'
            # Store the trimesh object in the nested dictionary
            nested_dict[nested_key] = mito_mesh
        except Exception as e:
            print(f'Error processing file {ply_file}: {e}')
    
    # Store the nested dictionary in the first-level dictionary
    trimesh_dictionary_by_cell[cell_key] = nested_dict

# Now, trimesh_dictionary_by_cell contains the organized trimesh objects


In [14]:
len(trimesh_dictionary_by_cell)

7

In [15]:
# Display the first 10 items in the nested dictionary

item_count = 0
for cell_key, nested_dict in trimesh_dictionary_by_cell.items():
    print(f'{cell_key}:')
    for nested_key, mito_mesh in nested_dict.items():
        print(f'  {nested_key}: {mito_mesh}')
        item_count += 1
        if item_count >= 10:
            break
    if item_count >= 10:
        break


cell_648518346349539076:
  trimesh_mito_888378: <trimesh.Trimesh(vertices.shape=(116, 3), faces.shape=(228, 3))>
  trimesh_mito_2421894: <trimesh.Trimesh(vertices.shape=(419, 3), faces.shape=(830, 3))>
  trimesh_mito_1561796: <trimesh.Trimesh(vertices.shape=(314, 3), faces.shape=(625, 3))>
  trimesh_mito_1041327: <trimesh.Trimesh(vertices.shape=(234, 3), faces.shape=(462, 3))>
  trimesh_mito_906019: <trimesh.Trimesh(vertices.shape=(123, 3), faces.shape=(223, 3))>
  trimesh_mito_1477121: <trimesh.Trimesh(vertices.shape=(66, 3), faces.shape=(128, 3))>
  trimesh_mito_906030: <trimesh.Trimesh(vertices.shape=(63, 3), faces.shape=(122, 3))>
  trimesh_mito_1936966: <trimesh.Trimesh(vertices.shape=(710, 3), faces.shape=(1414, 3))>
  trimesh_mito_1113138: <trimesh.Trimesh(vertices.shape=(141, 3), faces.shape=(278, 3))>
  trimesh_mito_852552: <trimesh.Trimesh(vertices.shape=(719, 3), faces.shape=(1432, 3))>


In [16]:
# Render all cells with the same settings and pull the viewpoint centroid from an index in cell_list (default index [0])

# Define colors and opacities
cell_color = [0.5, 0.5, 0.5]  # Neutral gray
cell_opac = 0.2

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

mito_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )

locals().update(mesh_actor)

# Creating a camera object and defining the view
if 'cell_' + str(cell_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(cell_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{cell_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000024FA3C76D10) at 0x0000024F9A6A18E8>

In [17]:
# Render all cells with the same settings and calculate a global mean of the viewpoint centroid

# Define colors and opacities
cell_color = [0.5, 0.5, 0.5]  # Neutral gray
cell_opac = 0.25

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

mito_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000024FA3C78000) at 0x0000024FB5882168>

In [18]:
# Render vasculature and all cells, using random cell colors and a range of cell opacities

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.025

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        # Randomize cell_color and cell_opac for each mesh
        cell_color = np.random.random(size=3)  # Random color
        cell_opac = np.random.uniform(0.1, 0.25)  # Random opacity between 0.005 and 0.05
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000024FA3C766C0) at 0x0000024FB5892A08>

In [19]:
# Render one cell and all the vasculature 

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.025

# Define the specific cell cell ID and its properties
specific_cell_id = cell_list[0]
cell_color = np.random.random(size=3)  # Random color
cell_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Add the specific cell mesh
cell_key = 'cell_' + str(specific_cell_id)
if cell_key in mesh_dictionary:
    mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
        mesh_dictionary[cell_key], 
        opacity=cell_opac, 
        color=cell_color
    )
    # Use the centroid of this specific cell mesh for the camera
    overall_centroid = mesh_dictionary[cell_key].centroid

# Add all the vasc meshes
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=150)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000024FA3C7A5E0) at 0x0000024FB58919A8>

In [20]:
# Render the vasc and one cell by querying the cell list

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.025

# User sets the index value
indx = 1  # Modify this index value as needed

# Get the specific cell ID from the cell_list
specific_cell_id = cell_list[indx]

# Define the specific cell cell properties
cell_color = np.random.random(size=3)  # Random color
cell_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Add the specific cell mesh
cell_key = 'cell_' + str(specific_cell_id)
if cell_key in mesh_dictionary:
    mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
        mesh_dictionary[cell_key], 
        opacity=cell_opac, 
        color=cell_color
    )
    # Use the centroid of this specific cell mesh for the camera
    overall_centroid = mesh_dictionary[cell_key].centroid

# Add all the vasc meshes
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=150)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000024FA3C78CA0) at 0x0000024FB587C3A8>

# render decimated mitochondria meshes

In [21]:
# Render vasc with all cell mitochondria
# cell cell bodies commented out
# even so, vtk view is heavily bogged down 

# Define colors and opacities
cell_color = [0.5, 0.5, 0.5]  # Neutral gray
cell_opac = 0.05

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.01

mito_opac = 0.75

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# uncomment to render cell bodies
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=cell_opac, 
            color=cell_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Creating a new dictionary to hold the mito mesh actors
mito_mesh_actor = {}

# Randomly assign colors to each group of cell meshes and add them to the actors
for cell_key, nested_dict in trimesh_dictionary_by_cell.items():
    # Generate a random color for this group of cell meshes
    cell_random_color = [random.random(), random.random(), random.random()]
    
    for mitoid, mito_mesh in nested_dict.items():
        # Define the key for the mesh actor
        actor_key = f'{cell_key}_trimesh_mito_{mitoid}'
        
        # Check if the loaded object is a Scene
        if isinstance(mito_mesh, trimesh.Scene):
            # Extract geometries from the Scene
            for name, geom in mito_mesh.geometry.items():
                mito_mesh = geom
                # Create the mesh actor with the random color and opacity
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=mito_opac, 
                    color=cell_random_color
                )
                centroids.append(mito_mesh.centroid)
        else:
            # Create the mesh actor with the random color and opacity
            mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                mito_mesh, 
                opacity=mito_opac, 
                color=cell_random_color
            )
            centroids.append(mito_mesh.centroid)

# Merge the mito mesh actors into the main mesh_actor dictionary
mesh_actor.update(mito_mesh_actor)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=300)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000025007A8FFE0) at 0x0000024FB5895108>

In [22]:
# save image to disk

import datetime

save_dir = 'vtk_images/'
base_name = 'all_cell_mito_with_vasc'
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

# camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=300) # 400 default
# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )

In [23]:
# render vasc and one queried cell with all its mitochondria

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.025

# Define the mito opacity
mito_opac = 0.75

# User sets the index value
indx = 1  # Modify this index value as needed

# Get the specific cell ID from the cell_list
specific_cell_id = cell_list[indx]
print(f"Index: {indx}, Specific cell ID: {specific_cell_id}")  # Debug statement

# Define the specific cell cell properties
cell_color = np.random.random(size=3)  # Random color
cell_opac = 0.2

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Add the specific cell mesh
cell_key = 'cell_' + str(specific_cell_id)
if cell_key in mesh_dictionary:
    mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
        mesh_dictionary[cell_key], 
        opacity=cell_opac, 
        color=cell_color
    )
    # Use the centroid of this specific cell mesh for the camera
    overall_centroid = mesh_dictionary[cell_key].centroid

# Add all the vasc meshes
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Creating a new dictionary to hold the mito mesh actors
mito_mesh_actor = {}

# Randomly assign colors to each group of cell meshes and add them to the actors
cell_key = f'cell_{specific_cell_id}'
if cell_key in trimesh_dictionary_by_cell:
    for mitoid, mito_mesh in trimesh_dictionary_by_cell[cell_key].items():
        # Generate a random color for this group of cell meshes
        cell_random_color = [random.random(), random.random(), random.random()]
        
        # Define the key for the mesh actor
        actor_key = f'{cell_key}_trimesh_mito_{mitoid}'
        
        # Check if the loaded object is a Scene
        if isinstance(mito_mesh, trimesh.Scene):
            # Extract geometries from the Scene
            for name, geom in mito_mesh.geometry.items():
                mito_mesh = geom
                # Create the mesh actor with the random color and opacity
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=mito_opac, 
                    color=cell_random_color
                )
                centroids.append(mito_mesh.centroid)
        else:
            # Create the mesh actor with the random color and opacity
            mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                mito_mesh, 
                opacity=mito_opac, 
                color=cell_random_color
            )
            centroids.append(mito_mesh.centroid)

# Merge the mito mesh actors into the main mesh_actor dictionary
mesh_actor.update(mito_mesh_actor)

locals().update(mesh_actor)

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=150)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


Index: 1, Specific cell ID: 648518346349538440
setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000025007A97E20) at 0x0000024F822A7BE8>

In [24]:
# save image to disk

import datetime

save_dir = 'vtk_images/'
base_name = f"cell_{specific_cell_id}_mito_with_vasc"
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

# camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=150) # 150 default
# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )